In [1]:
import spacy
import numpy as np
from sklearn.cluster import  OPTICS, DBSCAN,  AgglomerativeClustering
from sklearn.cluster._hdbscan.hdbscan import HDBSCAN

from pathlib import Path
from pysota.process import Persistence

nlp = spacy.load('en_core_web_lg')
results_dir = Path('../results/clean')

In [2]:
db = Persistence.load_files(results_dir)
print(len(db))

92


In [3]:
documents = [i.abstract for i in db]
doc_vectors = [nlp(doc).vector for doc in documents]
X = np.array(doc_vectors)

eps = .5
metric = 'euclidean'

# cluster = DBSCAN(eps=eps, min_samples=2)
# dbscan = OPTICS(min_samples=2)
cluster = AgglomerativeClustering(n_clusters=10, metric=metric, linkage='ward')
# cluster = HDBSCAN(metric='cosine',  max_cluster_size=20)
cluster.fit(X)

# Group documents by their cluster labels
clusters = {}
for idx, label in enumerate(cluster.labels_):
    clusters.setdefault(label, []).append(db[idx])

In [4]:
# Print the number of documents in each cluster
print(f' Number of clusters: {len(clusters)} \n\n')
for label, docs in clusters.items():
    if label == -1:
        print(f"Noise: {len(docs)} documents")
    else: 
        print(f"Cluster {label}: {len(docs)} documents")   

 Number of clusters: 10 


Cluster 4: 17 documents
Cluster 9: 6 documents
Cluster 1: 29 documents
Cluster 6: 8 documents
Cluster 5: 15 documents
Cluster 3: 9 documents
Cluster 7: 2 documents
Cluster 2: 3 documents
Cluster 0: 2 documents
Cluster 8: 1 documents


In [5]:
# Print the number of documents in each cluster
print(f' Number of clusters: {len(clusters)} with eps = {eps}\n\n')
for label, docs in clusters.items():
    if label == -1:
        print(f"Noise: {len(docs)} documents")
    else: 
        print(f"Cluster {label}: {len(docs)} documents")   
    for doc in docs:
        title = doc.title.replace('\n', ' ')
        print(f"  - {title}")
    print('\n\n ========================================================= \n\n')

 Number of clusters: 10 with eps = 0.5


Cluster 4: 17 documents
  - A survey of benchmarking frameworks for reinforcement learning
  - The Machine-Learning Approach of Reinforcement Learning
  - Quantum reinforcement learning
  - Reinforcement Learning: A Survey
  - Reinforcement Learning and Machine ethics:a systematic review
  - Topological Foundations of Reinforcement Learning
  - Motor Control and Reinforcement Learning
  - Local Explanations for Reinforcement Learning
  - Reinforcement Learning in Robotics
  - A Survey of Reinforcement Learning Techniques: Strategies, Recent Development, and Future Directions
  - Lexicographic Multi-Objective Reinforcement Learning
  - Reinforcement Learning and Causal Models
  - Reinforcement Learning
  - Statistical learning as reinforcement learning phenomena
  - Reinforcement Learning: Connections, Surprises, Challenges
  - The Reinforcement Learning Competitions
  - Determinantal Reinforcement Learning




Cluster 9: 6 documents
  - Computat

In [6]:
Persistence.save_clusters(clusters, Path(f'../results/clustered/{metric}'))